In [1]:
import calendar
import geopandas as gpd
import pandas as pd

In [2]:
# load shapefiles
buildings = gpd.read_file('Buildings.shp')
imp_surf = gpd.read_file('Impervious_Surface.shp')
buildings

,Shape_Leng,Shape_Area,Bldg_ID,geometry
0,327.296216,3639.258818,A,MULTIPOLYGON Z (((6182705.034 1992640.558 0.00...
1,378.700423,3594.692765,B,MULTIPOLYGON Z (((6182802.222 1992691.542 0.00...
2,422.995297,3781.848801,C,MULTIPOLYGON Z (((6182821.835 1992676.573 0.00...
3,274.203169,3310.957295,D,"POLYGON Z ((6182905.509 1992640.004 0.000, 618..."
4,334.633531,4113.261043,E,MULTIPOLYGON Z (((6182903.849 1992522.572 0.00...
5,298.293380,4060.215962,F,"POLYGON Z ((6182903.024 1992492.154 0.000, 618..."
6,367.651641,3598.945950,G,MULTIPOLYGON Z (((6182677.835 1992528.151 0.00...
7,336.173892,3466.562519,H,MULTIPOLYGON Z (((6182771.360 1992530.031 0.00...


In [3]:
# confirm coordinate system information
buildings.crs

<Derived Projected CRS: EPSG:6424>
Name: NAD83(2011) / California zone 5 (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - California - counties Kern; Los Angeles; San Bernardino; San Luis Obispo; Santa Barbara; Ventura.
- bounds: (-121.42, 32.76, -114.12, 35.81)
Coordinate Operation:
- name: SPCS83 California zone 5 (US Survey feet)
- method: Lambert Conic Conformal (2SP)
Datum: NAD83 (National Spatial Reference System 2011)
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [4]:
# first attempt

# filter buildings A through D
buildings_ad = buildings[buildings['Bldg_ID'].isin(['A', 'B', 'C', 'D'])]

# avg yearly rainfall by month in inches
average_year = [5.6, 5.9, 4.1, 1.1, 0.6, 0.1, 0.01, 0.01, 0.1, 0.9, 1.4, 3.3]

# constants for conversion
inches_to_feet = 12
cubic_feet_per_gallon = 7.48052

# monthly collection efficiency rate
collection_efficiency = 0.85

# calculate rainwater collection potential for each building
collection_potentials = []

for idx, row in buildings_ad.iterrows():
    building_name = row['Bldg_ID']
    collection_area_sqft = row['Shape_Area']
    
    collection_potential_avg = [
        round((collection_area_sqft * precip / inches_to_feet) * collection_efficiency * cubic_feet_per_gallon, 1)
        for precip in average_year
    ]
    
    # create a df to store the results
    data = {
        'Building': building_name,  
        'Month': [calendar.month_abbr[i] for i in range(1, 13)],
        'Average Water Year': collection_potential_avg
    }
    
    collection_df = pd.DataFrame(data)
    collection_potentials.append(collection_df)

# concatenate results for all buildings
final_collection = pd.concat(collection_potentials, ignore_index=True) 

#final_collection.to_csv('rainwater_collection_potential.csv')
#final_collection.to_csv('rainwater_collection_potential.csv', index=False)
final_collection

,Building,Month,Average Water Year
0,A,Jan,10798.7
1,A,Feb,11377.2
2,A,Mar,7906.2
3,A,Apr,2121.2
4,A,May,1157.0
5,A,Jun,192.8
6,A,Jul,19.3
7,A,Aug,19.3
8,A,Sep,192.8
9,A,Oct,1735.5


In [5]:
# all three scenarios with pivot tables

# filter buildings A through D
buildings_ad = buildings[buildings['Bldg_ID'].isin(['A', 'B', 'C', 'D'])]

# monthly rainfall by scenario in inches
scenarios = {
    'Average Water Year': [5.6, 5.9, 4.1, 1.1, 0.6, 0.1, 0.01, 0.01, 0.1, 0.9, 1.4, 3.3],
    'Dry Water Year': [3.9, 4.1, 2.8, 0.8, 0.4, 0.07, 0.007, 0.007, 0.07, 0.6, 0.9, 2.3],
    'Wet Water Year': [7.2, 7.6, 5.3, 1.4, 0.8, 0.13, 0.013, 0.013, 0.13, 1.17, 1.82, 4.29]
}

# constants for conversion and efficiency rate
inches_to_feet = 12
cubic_feet_per_gallon = 7.48052
collection_efficiency = 0.85

# calculate rainwater collection potential for each building and scenario
collection_potentials = []

for idx, row in buildings_ad.iterrows():
    building_name = row['Bldg_ID']
    collection_area_sqft = row['Shape_Area']
    
    building_potentials = []
    for scenario, rainfall in scenarios.items():
        collection_potential = [
            round((collection_area_sqft * precip / inches_to_feet) * collection_efficiency * cubic_feet_per_gallon, 1)
            for precip in rainfall
        ]
        building_potentials.append(collection_potential)
    
    data = {
        'Building': building_name,
        'Month': [calendar.month_abbr[i] for i in range(1, 13)]
    }
    for i, scenario in enumerate(scenarios.keys()):
        data[scenario] = building_potentials[i]
    
    building_df = pd.DataFrame(data)
    collection_potentials.append(building_df)

# concatenate results for all buildings
avg_dry_wet = pd.concat(collection_potentials, ignore_index=True)

# create pivot tables for average, dry, and wet scenarios
pivot_tables = {}
for scenario in scenarios.keys():
    pivot_table = avg_dry_wet.pivot_table(
        index='Building',
        columns=pd.Categorical(avg_dry_wet['Month'], categories=calendar.month_abbr[1:], ordered=True).codes + 1,
        values=scenario
    )
    pivot_table.columns.name = 'Month'
    pivot_table.columns = calendar.month_abbr[1:]
    pivot_tables[scenario] = pivot_table

# display tables
for scenario, pivot_table in pivot_tables.items():
    print(scenario + ' (in gallons):')
    display(pivot_table)

Average Water Year (in gallons):


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Building,,,,,,,,,,,,
A,10798.7,11377.2,7906.2,2121.2,1157.0,192.8,19.3,19.3,192.8,1735.5,2699.7,6363.5
B,10666.4,11237.9,7809.4,2095.2,1142.8,190.5,19.0,19.0,190.5,1714.2,2666.6,6285.6
C,11221.8,11822.9,8215.9,2204.3,1202.3,200.4,20.0,20.0,200.4,1803.5,2805.4,6612.8
D,9824.5,10350.8,7192.9,1929.8,1052.6,175.4,17.5,17.5,175.4,1578.9,2456.1,5789.4


Dry Water Year (in gallons):


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Building,,,,,,,,,,,,
A,7520.5,7906.2,5399.3,1542.7,771.3,135.0,13.5,13.5,135.0,1157.0,1735.5,4435.2
B,7428.4,7809.4,5333.2,1523.8,761.9,133.3,13.3,13.3,133.3,1142.8,1714.2,4380.9
C,7815.2,8215.9,5610.9,1603.1,801.6,140.3,14.0,14.0,140.3,1202.3,1803.5,4608.9
D,6842.1,7192.9,4912.3,1403.5,701.8,122.8,12.3,12.3,122.8,1052.6,1578.9,4035.1


Wet Water Year (in gallons):


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Building,,,,,,,,,,,,
A,13884.0,14655.3,10220.2,2699.7,1542.7,250.7,25.1,25.1,250.7,2256.2,3509.6,8272.6
B,13714.0,14475.9,10095.0,2666.6,1523.8,247.6,24.8,24.8,247.6,2228.5,3466.6,8171.3
C,14428.0,15229.6,10620.6,2805.4,1603.1,260.5,26.1,26.1,260.5,2344.5,3647.1,8596.7
D,12631.5,13333.3,9298.2,2456.1,1403.5,228.1,22.8,22.8,228.1,2052.6,3193.0,7526.3


In [6]:
# create a pivot table with ordered months
pivot_table = final_collection.pivot_table(
    index='Building',
    columns=pd.Categorical(final_collection['Month'], categories=calendar.month_abbr[1:], ordered=True).codes + 1,
    values='Average Water Year'
)

# rename the columns back to abbreviated month names
pivot_table.columns = calendar.month_abbr[1:]
pivot_table

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Building,,,,,,,,,,,,
A,10798.7,11377.2,7906.2,2121.2,1157.0,192.8,19.3,19.3,192.8,1735.5,2699.7,6363.5
B,10666.4,11237.9,7809.4,2095.2,1142.8,190.5,19.0,19.0,190.5,1714.2,2666.6,6285.6
C,11221.8,11822.9,8215.9,2204.3,1202.3,200.4,20.0,20.0,200.4,1803.5,2805.4,6612.8
D,9824.5,10350.8,7192.9,1929.8,1052.6,175.4,17.5,17.5,175.4,1578.9,2456.1,5789.4


In [ ]:
pivot_table.reset_index(inplace=True)
pivot_table.to_csv('Rainwater Collection Potential for Average Water Year (in gallons).csv', index=False)

In [ ]:
pivot_table_html = pivot_table.to_html()
print(pivot_table_html)

In [ ]:
# HTML table
html_content = """
<!DOCTYPE html>
<html>
<head>
    <style>
        table {
            border-collapse: collapse;
            width: 100%;
        }
        th, td {
            border: 1px solid black;
            padding: 8px;
            text-align: left;
        }
    </style>
</head>
<body>

<h2>Rainwater Collection Potential for Average Water Year (in gallons)</h2>

<table>
    <table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Jan</th>
      <th>Feb</th>
      <th>Mar</th>
      <th>Apr</th>
      <th>May</th>
      <th>Jun</th>
      <th>Jul</th>
      <th>Aug</th>
      <th>Sep</th>
      <th>Oct</th>
      <th>Nov</th>
      <th>Dec</th>
    </tr>
    <tr>
      <th>Building</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>A</th>
      <td>10798.7</td>
      <td>11377.2</td>
      <td>7906.2</td>
      <td>2121.2</td>
      <td>1157.0</td>
      <td>192.8</td>
      <td>19.3</td>
      <td>19.3</td>
      <td>192.8</td>
      <td>1735.5</td>
      <td>2699.7</td>
      <td>6363.5</td>
    </tr>
    <tr>
      <th>B</th>
      <td>10666.4</td>
      <td>11237.9</td>
      <td>7809.4</td>
      <td>2095.2</td>
      <td>1142.8</td>
      <td>190.5</td>
      <td>19.0</td>
      <td>19.0</td>
      <td>190.5</td>
      <td>1714.2</td>
      <td>2666.6</td>
      <td>6285.6</td>
    </tr>
    <tr>
      <th>C</th>
      <td>11221.8</td>
      <td>11822.9</td>
      <td>8215.9</td>
      <td>2204.3</td>
      <td>1202.3</td>
      <td>200.4</td>
      <td>20.0</td>
      <td>20.0</td>
      <td>200.4</td>
      <td>1803.5</td>
      <td>2805.4</td>
      <td>6612.8</td>
    </tr>
    <tr>
      <th>D</th>
      <td>9824.5</td>
      <td>10350.8</td>
      <td>7192.9</td>
      <td>1929.8</td>
      <td>1052.6</td>
      <td>175.4</td>
      <td>17.5</td>
      <td>17.5</td>
      <td>175.4</td>
      <td>1578.9</td>
      <td>2456.1</td>
      <td>5789.4</td>
    </tr>
  </tbody>
</table>
</table>

</body>
</html>
"""

with open("html_table.html", "w") as file:
    file.write(html_content)

In [7]:
# precipitation data (in inches) for different scenarios
average_year = [5.6, 5.9, 4.1, 1.1, 0.6, 0.1, 0.01, 0.01, 0.1, 0.9, 1.4, 3.3]
dry_year = [3.9, 4.1, 2.8, 0.8, 0.4, 0.07, 0.007, 0.007, 0.07, 0.6, 0.9, 2.3]
wet_year = [7.2, 7.6, 5.3, 1.4, 0.8, 0.13, 0.013, 0.013, 0.13, 1.17, 1.82, 4.29]

# constants for conversion
inches_to_feet = 12
collection_efficiency = 0.85
cubic_feet_per_gallon = 7.48052

# calculate yearly rainwater collection potential for each scenario
def calculate_yearly_total(precipitation):
    collection_area_sqft = imp_surf['Shape_Area'].sum()
    return round((collection_area_sqft * sum(precipitation) / inches_to_feet) * collection_efficiency * cubic_feet_per_gallon, 1)

yearly_total_average = calculate_yearly_total(average_year)
yearly_total_dry = calculate_yearly_total(dry_year)
yearly_total_wet = calculate_yearly_total(wet_year)

print(f'Annual Collection Potential (Average Year): {yearly_total_average:.1f} gallons')
print(f'Annual Collection Potential (Dry Year): {yearly_total_dry:.1f} gallons')
print(f'Annual Collection Potential (Wet Year): {yearly_total_wet:.1f} gallons')

Annual Collection Potential (Average Year): 414309.1 gallons
Annual Collection Potential (Dry Year): 285894.8 gallons
Annual Collection Potential (Wet Year): 535197.0 gallons


In [ ]:
import plotly.express as px

# plotly line chart
fig = px.line(final_collection, x='Month', y='Average Water Year',
              color='Building', title='Rainwater Collection Potential for Average Water Year')

fig.update_layout(xaxis_title='Month',
                  yaxis_title='Collection Potential',
                  legend_title='Building')
fig.show()